# Deploying an app again

## Deeper Dive - I Think SO

In [1]:
%env k=microk8s.kubectl

env: k=microk8s.kubectl


In [2]:
! $k create deployment hello-node --image=gcr.io/hello-minikube-zero-install/hello-node

Error from server (AlreadyExists): deployments.apps "hello-node" already exists


In [3]:
! $k get all

NAME                              READY   STATUS    RESTARTS   AGE
pod/hello-node-7676b5fb8d-7dncj   1/1     Running   0          10m

NAME                 TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
service/hello-node   LoadBalancer   10.152.183.50   <pending>     80:32584/TCP   7m50s
service/kubernetes   ClusterIP      10.152.183.1    <none>        443/TCP        14h

NAME                         READY   UP-TO-DATE   AVAILABLE   AGE
deployment.apps/hello-node   1/1     1            1           10m

NAME                                    DESIRED   CURRENT   READY   AGE
replicaset.apps/hello-node-7676b5fb8d   1         1         1       10m


In [4]:
! $k get events

LAST SEEN   TYPE     REASON              OBJECT                             MESSAGE
10m         Normal   Scheduled           pod/hello-node-7676b5fb8d-7dncj    Successfully assigned default/hello-node-7676b5fb8d-7dncj to k8s
10m         Normal   Pulling             pod/hello-node-7676b5fb8d-7dncj    Pulling image "gcr.io/hello-minikube-zero-install/hello-node"
10m         Normal   Pulled              pod/hello-node-7676b5fb8d-7dncj    Successfully pulled image "gcr.io/hello-minikube-zero-install/hello-node"
10m         Normal   Created             pod/hello-node-7676b5fb8d-7dncj    Created container hello-node
10m         Normal   Started             pod/hello-node-7676b5fb8d-7dncj    Started container hello-node
10m         Normal   SuccessfulCreate    replicaset/hello-node-7676b5fb8d   Created pod: hello-node-7676b5fb8d-7dncj
10m         Normal   ScalingReplicaSet   deployment/hello-node              Scaled up replica set hello-node-7676b5fb8d to 1


### To make the Pod accessible to the world outside the K8s cluster, 

- you need to expose the Pod as a Service. Create a Service with the command 

## Expose it!

In [5]:
! $k expose deployment hello-node --type=LoadBalancer --port=80

Error from server (AlreadyExists): services "hello-node" already exists


In [6]:
! $k get services

NAME         TYPE           CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
hello-node   LoadBalancer   10.152.183.50   <pending>     80:32584/TCP   7m51s
kubernetes   ClusterIP      10.152.183.1    <none>        443/TCP        14h


In [7]:
! curl 10.152.183.50  ## need to remember why

curl: (7) Failed to connect to 10.152.183.50 port 80: Connection refused


In [8]:
! curl localhost 

curl: (7) Failed to connect to localhost port 80: Connection refused


In [9]:
# ! curl 10.152.183.50:32584

In [10]:
! curl 10.12.69.17:80

curl: (7) Failed to connect to 10.12.69.17 port 80: Connection refused


In [12]:
! netstat -anlp | grep 80

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)
tcp        0      0 127.0.0.1:2380          0.0.0.0:*               LISTEN      -                   
tcp        0      0 0.0.0.0:8080            0.0.0.0:*               LISTEN      27433/python3       
tcp        0      0 10.12.69.17:8080        158.68.216.171:27094    ESTABLISHED 27433/python3       
tcp        0      0 127.0.0.1:44845         127.0.0.1:49080         ESTABLISHED 2018/python3        
tcp        0   1251 10.12.69.17:8080        158.68.216.171:1923     ESTABLISHED 27433/python3       
tcp        0      0 10.1.68.1:58700         10.1.68.6:8000          ESTABLISHED -                   
tcp        0      0 127.0.0.1:49080         127.0.0.1:44845         ESTABLISHED 27433/python3       
tcp        0      0 10.12.69.17:8080        158.68.216.171:9793     ESTABLISHED 27433/python3       
tcp        0      0 127.0.0.1:60080         127.0.0.1:12379        

In [14]:
! $k delete service hello-node

service "hello-node" deleted


In [15]:
! $k get services

NAME         TYPE        CLUSTER-IP     EXTERNAL-IP   PORT(S)   AGE
kubernetes   ClusterIP   10.152.183.1   <none>        443/TCP   15h


In [16]:
# try again
! $k create deployment hello-node --image=gcr.io/hello-minikube-zero-install/hello-node
! $k expose deployment hello-node --type=LoadBalancer --port=7777

Error from server (AlreadyExists): deployments.apps "hello-node" already exists
service/hello-node exposed


In [17]:
! netstat -anlp |grep 7777

(Not all processes could be identified, non-owned process info
 will not be shown, you would have to be root to see it all.)


https://stackoverflow.com/questions/57530575/how-to-expose-microk8s-containers-so-they-are-available-from-another-machine

You are right, if you want to connect to your cluster from outside, you have to expose service as LoadBalancer or NodePort.

Unfortunately, as you are not using any cloud provider which support LoadBalancer (like AWS or GCP which provides External-IPs) you will be not able to expose service as LoadBalancer (service stuck on Pending state).

https://itnext.io/an-illustrated-guide-to-kubernetes-networking-part-1-d1ede3322727
>Even though there are a bunch of tools available to setup and manage a cluster, you’ve still wondered how it all works under the hood. And where do you look if it breaks? I know I did.

Sure Kubernetes is simple enough to start using it. But let’s face it — 

## it’s a complex beast under the hood. 
### There are a lot of moving parts, and knowing how they all fit in and work together is a must, 

 > if you want to be ready for failures. One of the most complex, and probably the most critical parts is the Networking.
 
### Microk8s is not the best for this - we will work on this when we have a full cluster 


## So we just introduce networking here

# Every Pod hs a unique IP

![nodeport](https://miro.medium.com/max/1013/0*QtFlPHXtuol02b_j.png)

In [18]:
! $k delete service hello-node

service "hello-node" deleted


In [19]:
! $k create deployment hello-node --image=gcr.io/hello-minikube-zero-install/hello-node
! $k expose deployment hello-node --type=NodePort --port=80

Error from server (AlreadyExists): deployments.apps "hello-node" already exists
service/hello-node exposed


In [20]:
!$k get services

NAME         TYPE        CLUSTER-IP      EXTERNAL-IP   PORT(S)        AGE
hello-node   NodePort    10.152.183.57   <none>        80:32487/TCP   20s
kubernetes   ClusterIP   10.152.183.1    <none>        443/TCP        15h


In [21]:
! curl localhost

curl: (7) Failed to connect to localhost port 80: Connection refused


In [22]:
! curl 10.152.183.57

curl: (7) Failed to connect to 10.152.183.57 port 80: Connection refused


In [1]:
# ! curl 10.152.183.57:32487 # runs forevere

In [2]:
! $k get services

/bin/sh: 1: get: not found
